# Tratamento de Dados

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

db_path = os.path.abspath("../../data/brasileirao.db")

engine = create_engine(f"sqlite:///{db_path}")

df = pd.read_sql_query("SELECT * FROM jogos_historico", con=engine)


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4180 entries, 0 to 4179
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          4180 non-null   int64 
 1   temporada   4180 non-null   int64 
 2   rodada      4180 non-null   object
 3   dia_semana  4180 non-null   object
 4   data        4180 non-null   object
 5   hora        4180 non-null   object
 6   mandante    4180 non-null   object
 7   visitante   4180 non-null   object
 8   placar      4180 non-null   object
 9   publico     3489 non-null   object
 10  estadio     4179 non-null   object
dtypes: int64(2), object(9)
memory usage: 359.3+ KB


In [3]:
df.head()

,id,temporada,rodada,dia_semana,data,hora,mandante,visitante,placar,publico,estadio
0,1,2014,1,sáb,2014-04-19,18:30:00.000000,Fluminense,Figueirense,3–0,35.020,Estadio Jornalista Mário Filho
1,2,2014,1,sáb,2014-04-19,18:30:00.000000,Internacional,Vitória,1–0,21.983,Estádio José Pinheiro Borba
2,3,2014,1,dom,2014-04-20,16:00:00.000000,Atlético Mineiro,Corinthians,0–0,8.724,Estádio Municipal João Havelange
3,4,2014,1,dom,2014-04-20,16:00:00.000000,São Paulo,Botafogo (RJ),3–0,31.564,Estádio do Morumbi
4,5,2014,1,dom,2014-04-20,16:00:00.000000,Bahia,Cruzeiro,1–2,9.348,Itaipava Arena Fonte Nova


In [4]:
df["data"] = pd.to_datetime(df["data"], errors = "coerce")
df['data_hora'] = pd.to_datetime(df['data'].dt.strftime('%Y-%m-%d') + ' ' + df['hora'].astype(str), errors='coerce')


In [5]:
df['rodada'] = pd.to_numeric(df['rodada'], errors='coerce').astype('Int64')

In [6]:
df["publico"] = (df["publico"].astype(str).str.replace(",", "").str.replace(".", ""))
df["publico"] = pd.to_numeric(df["publico"], errors = "coerce").astype("Int64")

In [7]:
import re

# Extrair gols do placar
def extrair_gols(placar):
    if isinstance(placar, str):
        placar = placar.strip()
        # Substituir travessão por 'x' ou '-' para padronizar
        placar = placar.replace('\u2013', 'x').replace('–', 'x')  # Adicional para garantir
        if re.match(r'^\d+x\d+$', placar):
            gols = placar.split('x')
            return int(gols[0]), int(gols[1])
    return None, None

df[['gols_mandante', 'gols_visitante']] = df['placar'].apply(
    lambda x: pd.Series(extrair_gols(x))
)

df['gols_mandante'] = df['gols_mandante'].astype('Int64')
df['gols_visitante'] = df['gols_visitante'].astype('Int64')

df['total_gols'] = (df['gols_mandante'] + df['gols_visitante']).astype('Int64')


In [8]:
def resultado_partida(row):
    if pd.isna(row['gols_mandante']) or pd.isna(row['gols_visitante']):
        return None
    if row['gols_mandante'] > row['gols_visitante']:
        return 'mandante'
    elif row['gols_mandante'] < row['gols_visitante']:
        return 'visitante'
    else:
        return 'empate'

df['resultado_partida'] = df.apply(resultado_partida, axis=1)

In [9]:
colunas_modelo = [
    'temporada', 'rodada', 'dia_semana', 'data_hora', 'estadio',
    'publico', 'mandante', 'visitante', 'gols_mandante',
    'gols_visitante', 'total_gols', 'resultado_partida'
]

df_modelo = df[colunas_modelo]

In [10]:
df_modelo.head()

,temporada,rodada,dia_semana,data_hora,estadio,publico,mandante,visitante,gols_mandante,gols_visitante,total_gols,resultado_partida
0,2014,1,sáb,2014-04-19 18:30:00,Estadio Jornalista Mário Filho,35020,Fluminense,Figueirense,3,0,3,mandante
1,2014,1,sáb,2014-04-19 18:30:00,Estádio José Pinheiro Borba,21983,Internacional,Vitória,1,0,1,mandante
2,2014,1,dom,2014-04-20 16:00:00,Estádio Municipal João Havelange,8724,Atlético Mineiro,Corinthians,0,0,0,empate
3,2014,1,dom,2014-04-20 16:00:00,Estádio do Morumbi,31564,São Paulo,Botafogo (RJ),3,0,3,mandante
4,2014,1,dom,2014-04-20 16:00:00,Itaipava Arena Fonte Nova,9348,Bahia,Cruzeiro,1,2,3,visitante


In [11]:
# Remover linhas com estádio nulo
df_modelo = df_modelo.dropna(subset=['estadio']).reset_index(drop=True)

print(f"Total de partidas após remoção de estádio nulo: {df_modelo.shape[0]}")

Total de partidas após remoção de estádio nulo: 4179


In [12]:
# Mostrar apenas jogos onde o público é nulo
df_publico_nulo = df_modelo[df_modelo['publico'].isna()]

print(f"Total de jogos com público nulo: {df_publico_nulo.shape[0]}")
display(df_publico_nulo[['temporada', 'rodada', 'data_hora', 'mandante', 'visitante', 'estadio', 'publico']])

Total de jogos com público nulo: 691


,temporada,rodada,data_hora,mandante,visitante,estadio,publico
107,2014,11,2014-07-20 18:30:00,Ath Paranaense,Criciúma,Estádio Joaquim Américo Guimarães,<NA>
115,2014,12,2014-07-27 16:00:00,Ath Paranaense,Fluminense,Estádio Joaquim Américo Guimarães,<NA>
132,2014,14,2014-08-10 16:00:00,Ath Paranaense,Botafogo (RJ),Estádio Joaquim Américo Guimarães,<NA>
168,2014,17,2014-08-24 18:30:00,Ath Paranaense,Bahia,Estádio Joaquim Américo Guimarães,<NA>
186,2014,19,2014-09-07 18:30:00,Bahia,Coritiba,Itaipava Arena Fonte Nova,<NA>
...,...,...,...,...,...,...,...
3792,2023,38,2023-12-06 21:30:00,Coritiba,Corinthians,Estádio Major Antônio Couto Pereira,<NA>
4137,2024,34,2024-11-20 21:30:00,Atlético Mineiro,Botafogo (RJ),Estádio Raimundo Sampaio,<NA>
4149,2024,36,2024-11-26 21:30:00,Atlético Mineiro,Juventude,Estádio Raimundo Sampaio,<NA>
4168,2024,37,2024-12-04 21:30:00,Cruzeiro,Palmeiras,Estádio Governador Magalhães Pinto,<NA>


In [13]:
df_modelo['publico'] = df_modelo['publico'].fillna(0).astype('Int64')

# Análise Exploratória

In [14]:
import plotly.express as px

print(f"Total de partidas: {df_modelo.shape[0]}")
print("\n")
print(df_modelo.info())
print("\n")
print(df_modelo.isna().sum())

Total de partidas: 4179


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4179 entries, 0 to 4178
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   temporada          4179 non-null   int64         
 1   rodada             4179 non-null   Int64         
 2   dia_semana         4179 non-null   object        
 3   data_hora          4179 non-null   datetime64[ns]
 4   estadio            4179 non-null   object        
 5   publico            4179 non-null   Int64         
 6   mandante           4179 non-null   object        
 7   visitante          4179 non-null   object        
 8   gols_mandante      4179 non-null   Int64         
 9   gols_visitante     4179 non-null   Int64         
 10  total_gols         4179 non-null   Int64         
 11  resultado_partida  4179 non-null   object        
dtypes: Int64(5), datetime64[ns](1), int64(1), object(5)
memory usage: 412.3+ KB
None


temporada

In [15]:
# --- Distribuição dos resultados da partida ---
fig_resultado = px.histogram(
    df_modelo,
    x="resultado_partida",
    color="resultado_partida",
    title="Distribuição dos Resultados (Mandante / Empate / Visitante)",
    text_auto=True,
    color_discrete_map={
        "mandante": "blue",
        "visitante": "red",
        "empate": "gray"
    }
)
fig_resultado.show()

In [16]:
# --- Distribuição do total de gols ---
fig_gols = px.histogram(
    df_modelo,
    x="total_gols",
    title="Distribuição do Total de Gols por Partida",
    text_auto=True,
    category_orders={"total_gols": sorted(df_modelo['total_gols'].unique())}
)
# Corrigir espaçamento dos ticks
fig_gols.update_layout(
    xaxis=dict(
        tickmode='linear',
        tick0=0,
        dtick=1
    ),
    bargap=0.1
)

fig_gols.show()

In [17]:
fig_dia_semana = px.histogram(
    df_modelo,
    x="dia_semana",
    title="Número de Jogos por Dia da Semana",
    text_auto=True,
    color_discrete_sequence=["green"]
)

fig_dia_semana.show()

In [18]:
# Público médio por temporada
publico_medio_temporada = df_modelo.groupby('temporada')['publico'].mean().reset_index()

fig_publico_temporada = px.bar(
    publico_medio_temporada,
    x='temporada',
    y='publico',
    title="Público Médio por Temporada",
    text_auto='.2s',
    color_discrete_sequence=["orange"]
)

fig_publico_temporada.update_layout(
    xaxis=dict(tickmode='linear', dtick=1),
    bargap=0.3
)

fig_publico_temporada.show()

In [19]:
# Vamos pegar só a hora (sem minutos/segundos) para análise
df_modelo['hora_hora'] = df_modelo['data_hora'].dt.hour

fig_horarios = px.histogram(
    df_modelo,
    x="hora_hora",
    nbins=20,
    title="Distribuição dos Horários dos Jogos",
    text_auto=True,
    color_discrete_sequence=["teal"]
)

fig_horarios.update_layout(
    xaxis=dict(tickmode='linear', dtick=1),
    bargap=0.2
)

fig_horarios.show()


In [20]:
# Criar uma coluna indicando quem foi o vencedor
def determinar_vencedor(row):
    if row['resultado_partida'] == 'mandante':
        return row['mandante']
    elif row['resultado_partida'] == 'visitante':
        return row['visitante']
    else:
        return None

# Aplicar a função para criar coluna de vencedor
df_modelo['vencedor'] = df_modelo.apply(determinar_vencedor, axis=1)

# Contar vitórias por time
vitorias_totais = df_modelo['vencedor'].value_counts().reset_index()
vitorias_totais.columns = ['time', 'vitorias_totais']

# Top 10 times que mais venceram
fig_vitorias_totais = px.bar(
    vitorias_totais.head(10),
    x='time',
    y='vitorias_totais',
    title="Top 10 Times com Mais Vitórias (Mandante + Visitante)",
    text_auto=True,
    color_discrete_sequence=["mediumseagreen"]
)

fig_vitorias_totais.update_layout(
    xaxis_title="Time",
    yaxis_title="Vitórias",
    xaxis_tickangle=-45
)

fig_vitorias_totais.show()


In [21]:
# Filtrar apenas vitórias do mandante
vitorias_mandante = df_modelo[df_modelo['resultado_partida'] == 'mandante']

# Contar vitórias por time mandante
vitorias_por_time = vitorias_mandante['mandante'].value_counts().reset_index()
vitorias_por_time.columns = ['time', 'vitorias_mandante']

# Top 10 times que mais venceram em casa
fig_vitorias_mandante = px.bar(
    vitorias_por_time.head(10),
    x='time',
    y='vitorias_mandante',
    title="Top 10 Times com Mais Vitórias em Casa",
    text_auto=True,
    color_discrete_sequence=["blue"]
)

fig_vitorias_mandante.update_layout(
    xaxis_title="Time Mandante",
    yaxis_title="Vitórias",
    xaxis_tickangle=-45
)

fig_vitorias_mandante.show()


In [22]:
# Filtrar apenas vitórias do visitante
vitorias_visitante = df_modelo[df_modelo['resultado_partida'] == 'visitante']

# Contar vitórias como visitante
vitorias_visitante_por_time = vitorias_visitante['visitante'].value_counts().reset_index()
vitorias_visitante_por_time.columns = ['time', 'vitorias_como_visitante']

# Top 10 times com mais vitórias fora de casa
fig_vitorias_visitante = px.bar(
    vitorias_visitante_por_time.head(10),
    x='time',
    y='vitorias_como_visitante',
    title="Top 10 Times com Mais Vitórias como Visitante",
    text_auto=True,
    color_discrete_sequence=["darkred"]
)

fig_vitorias_visitante.update_layout(
    xaxis_title="Time Visitante",
    yaxis_title="Vitórias como Visitante",
    xaxis_tickangle=-45
)

fig_vitorias_visitante.show()


In [23]:
# Criar coluna de gols marcados por time mandante e visitante
gols_marcados_mandante = df_modelo[['mandante', 'gols_mandante']].rename(columns={'mandante': 'time', 'gols_mandante': 'gols'})
gols_marcados_visitante = df_modelo[['visitante', 'gols_visitante']].rename(columns={'visitante': 'time', 'gols_visitante': 'gols'})

# Concatenar e somar
gols_totais = pd.concat([gols_marcados_mandante, gols_marcados_visitante])
gols_totais = gols_totais.groupby('time')['gols'].sum().reset_index()
gols_totais = gols_totais.sort_values(by='gols', ascending=False)

# Top 10 times que mais marcaram gols
fig_gols_totais = px.bar(
    gols_totais.head(10),
    x='time',
    y='gols',
    title="Top 10 Times que Mais Marcaram Gols (Mandante + Visitante)",
    text_auto=True,
    color_discrete_sequence=["orangered"]
)

fig_gols_totais.update_layout(
    xaxis_title="Time",
    yaxis_title="Gols Marcados",
    xaxis_tickangle=-45
)

fig_gols_totais.show()


In [24]:
# Somar gols marcados em casa por time
gols_mandante_por_time = df_modelo.groupby('mandante')['gols_mandante'].sum().reset_index()
gols_mandante_por_time = gols_mandante_por_time.sort_values(by='gols_mandante', ascending=False)

# Top 10 times que mais marcaram gols em casa
fig_gols_mandante = px.bar(
    gols_mandante_por_time.head(10),
    x='mandante',
    y='gols_mandante',
    title="Top 10 Times que Mais Marcaram Gols em Casa",
    text_auto=True,
    color_discrete_sequence=["crimson"]
)

fig_gols_mandante.update_layout(
    xaxis_title="Time Mandante",
    yaxis_title="Gols Marcados",
    xaxis_tickangle=-45
)

fig_gols_mandante.show()


In [25]:
# Somar gols marcados como visitante
gols_visitante_por_time = df_modelo.groupby('visitante')['gols_visitante'].sum().reset_index()
gols_visitante_por_time = gols_visitante_por_time.sort_values(by='gols_visitante', ascending=False)

# Top 10 times que mais marcaram gols fora de casa
fig_gols_visitante = px.bar(
    gols_visitante_por_time.head(10),
    x='visitante',
    y='gols_visitante',
    title="Top 10 Times que Mais Marcaram Gols como Visitante",
    text_auto=True,
    color_discrete_sequence=["deepskyblue"]
)

fig_gols_visitante.update_layout(
    xaxis_title="Time Visitante",
    yaxis_title="Gols Marcados como Visitante",
    xaxis_tickangle=-45
)

fig_gols_visitante.show()
